In [1]:
import sys
import os
sys.path.append(os.path.dirname(os.getcwd()))

import logging
logging.getLogger().setLevel(logging.INFO)

import copy
import multiprocessing as mp

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import dash
import dash_bootstrap_components as dbc
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State
import plotly.express as px
import plotly.graph_objects as go
from jupyter_dash import JupyterDash

from config import config
from pof.loader.asset_model_loader import AssetModelLoader
from pof import Component, FailureMode, Task
from pof.interface.dashlogger import DashLogger
from pof.interface.layouts import *
from pof.interface.figures import update_condition_fig, update_cost_fig, update_pof_fig, make_inspection_interval_fig

In [2]:
comp = Component.demo()
fm = FailureMode.demo()

In [ ]:
comp.mc_timeline(200)

In [ ]:
pof = dict(
    maint=pd.DataFrame(comp.expected_pof(t_end=200)),
    no_maint=pd.DataFrame(comp.expected_untreated(t_end=200)),
)
pof

In [ ]:
pof = dict(
    maint=pd.DataFrame(local.expected_pof(t_end=200)),
    no_maint=pd.DataFrame(local.expected_untreated(t_end=200)),
)

# OLD CODE
# df = pd.concat(pof).rename_axis(["strategy", "time"]).reset_index()
# df.index.names = ["strategy", "key"]
# df = df.rename(columns={"variable": "source"})

# df = df.melt(
#     id_vars=["time", "strategy", "fm_active"],
#     var_name="source",
#     value_name="pof",
# )

df = pd.concat(pof).rename_axis(["strategy", "key"]).reset_index()

df_pof = df[df["key"] == "pof"]
df_pof = pd.melt(
    df_pof, id_vars=["strategy"], value_vars=df_pof.columns[2:]
).rename(columns={"variable": "source", "value": "pof"})
df_pof = df_pof.explode("pof", ignore_index=True)

df_active = df[df["key"] == "fm_active"]
df_active = pd.melt(
    df_active, id_vars=["strategy"], value_vars=df_active.columns[2:]
).rename(columns={"variable": "source", "value": "fm_active"})

df_time = df[df["key"] == "time"]
df_time = pd.melt(
    df_time, id_vars=["strategy"], value_vars=df_time.columns[2:]
).rename(columns={"variable": "source", "value": "time"})
df_time = df_time.explode("time", ignore_index=True)["time"]

df = df_pof.merge(df_active, on=["strategy", "source"])
df["time"] = df_time

df.sort_values(by=["source"], inplace=True)

color_map = get_color_map(df=df, column="source")

df = df[df["fm_active"] == True]

fig = px.line(
    df,
    x="time",
    y="pof",
    color="source",
    color_discrete_map=color_map,
    line_dash="strategy",
    line_group="strategy",
    title="Probability of Failure given Maintenance Strategy",
)

In [3]:
comp.mc_timeline(1000)
update_cost_fig(comp)

100%|██████████| 100/100 [00:02<00:00, 36.46it/s]


In [4]:
comp.mc_timeline(1000)
update_condition_fig(comp)

100%|██████████| 100/100 [00:05<00:00, 17.05it/s]


In [4]:
comp.mc_timeline(10)
make_inspection_interval_fig_drop(comp)

100%|██████████| 10/10 [00:00<00:00, 76.91it/s]


In [5]:
comp.mc_timeline(1000)
update_pof_fig(comp)

100%|██████████| 100/100 [00:05<00:00, 18.35it/s]


In [ ]:

def make_inspection_interval_fig_drop(
    local, t_min=0, t_max=10, step=1, n_iterations=10
):

    try:
        df = local.expected_inspection_interval(
            t_min=t_min, t_max=t_max, step=step, n_iterations=n_iterations
        )
        df = df.set_index("inspection_interval", drop=True)

        fig = go.Figure()

        for column in df.columns.to_list():
            fig.add_trace(
                go.Scatter(
                    x=df.index,
                    y=df[column],
                    name=column,
                    showlegend=False,
                )
            )

        fig.update_yaxes(automargin=True)
        fig.update_xaxes(automargin=True)

        # Add dropdown
        fig.update_layout(
            title="Risk v Cost at different Inspection Intervals",
            updatemenus=[
                go.layout.Updatemenu(
                    active=0,
                    buttons=list(
                        [
                            dict(
                                label="All",
                                method="update",
                                args=[
                                    {
                                        "visible": [
                                            True,
                                            True,
                                            True,
                                            True,
                                            True,
                                            True,
                                            True,
                                        ]
                                    },
                                    {
                                        "title": "Risk v Cost at different Inspection Intervals",
                                        "showlegend": False,
                                    },
                                ],
                            ),
                            dict(
                                label="Inspection",
                                method="update",
                                args=[
                                    {
                                        "visible": [
                                            True,
                                            False,
                                            False,
                                            False,
                                            False,
                                            False,
                                            False,
                                        ]
                                    },  # the index of True aligns with the indices of plot traces
                                    {
                                        "title": "Risk v Cost at different Inspection Intervals",
                                        "showlegend": False,
                                    },
                                ],
                            ),
                            dict(
                                label="On Condition Repair",
                                method="update",
                                args=[
                                    {
                                        "visible": [
                                            False,
                                            True,
                                            False,
                                            False,
                                            False,
                                            False,
                                            False,
                                        ]
                                    },
                                    {
                                        "title": "Risk v Cost at different Inspection Intervals",
                                        "showlegend": False,
                                    },
                                ],
                            ),
                            dict(
                                label="On Condition Replacement",
                                method="update",
                                args=[
                                    {
                                        "visible": [
                                            False,
                                            False,
                                            True,
                                            False,
                                            False,
                                            False,
                                            False,
                                        ]
                                    },
                                    {
                                        "title": "Risk v Cost at different Inspection Intervals",
                                        "showlegend": False,
                                    },
                                ],
                            ),
                            dict(
                                label="On Failure Replacement",
                                method="update",
                                args=[
                                    {
                                        "visible": [
                                            False,
                                            False,
                                            False,
                                            True,
                                            False,
                                            False,
                                            False,
                                        ]
                                    },
                                    {
                                        "title": "Risk v Cost at different Inspection Intervals",
                                        "showlegend": False,
                                    },
                                ],
                            ),
                            dict(
                                label="Risk Cost",
                                method="update",
                                args=[
                                    {
                                        "visible": [
                                            False,
                                            False,
                                            False,
                                            False,
                                            True,
                                            False,
                                            False,
                                        ]
                                    },
                                    {
                                        "title": "Risk v Cost at different Inspection Intervals",
                                        "showlegend": False,
                                    },
                                ],
                            ),
                            dict(
                                label="Direct Cost",
                                method="update",
                                args=[
                                    {
                                        "visible": [
                                            False,
                                            False,
                                            False,
                                            False,
                                            False,
                                            True,
                                            False,
                                        ]
                                    },
                                    {
                                        "title": "Risk v Cost at different Inspection Intervals",
                                        "showlegend": False,
                                    },
                                ],
                            ),
                            dict(
                                label="Total",
                                method="update",
                                args=[
                                    {
                                        "visible": [
                                            False,
                                            False,
                                            False,
                                            False,
                                            False,
                                            False,
                                            True,
                                        ]
                                    },
                                    {
                                        "title": "Risk v Cost at different Inspection Intervals",
                                        "showlegend": False,
                                    },
                                ],
                            ),
                        ]
                    ),
                    direction="down",
                    pad={"r": 10, "t": 10},
                    showactive=True,
                    x=0.5,
                    xanchor="right",
                    y=1.2,
                    yanchor="top",
                ),
            ],
        )

    except:
        fig = go.Figure(
            layout=go.Layout(
                title=go.layout.Title(text="Error Producing Inspection Interval")
            )
        )

    return fig

In [12]:
cond_name = "cond name"
"".join([x[0] for x in cond_name.split(" ")])

'cn'